In [1]:
import sys
sys.path.append("/Users/mattkollada/openai_gym/pendulum/")

import gym
import torch

from models import *
from gym_util import *
from data import *
from agent import *

In [2]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [3]:
env = gym.make( 'CartPole-v1' )


In [4]:
episodes_per_epoch = 100
epochs_per_update = 5
rand_prob = 0.1
discount_rate = 0.95

num_runs = 100
num_epochs = 5

timesteps_per_run = 500

lr=0.001

In [5]:
dq_net = FullyConnected( 4, [16,2], dropout=0 )
dq_agent = DeepQAgent( env.action_space, env.observation_space, dq_net,
                      discount_rate=discount_rate, rand_prob=rand_prob )

criterion = nn.MSELoss()
optimizer = torch.optim.Adam( dq_agent.prediction_model.parameters(), lr=lr )

In [6]:
dq_agent.prediction_model = torch.load( './models/best_model_483.945-avg.pkl' )
dq_agent.update_target_model()

In [7]:
dq_agent.rand_prob = 0
x = run_episode( env, dq_agent, render=True, timesteps=10000 )
env.close()

In [32]:
max_time_steps = 0

In [33]:
dq_agent.discount_rate

0.95

In [8]:
# dq_agent.rand_prob = 0.1
# dq_agent.discount_rate = 0.95

test_data = {}
for run in range( num_runs ):
    cartpole_data = run_n_episodes( env, dq_agent, episodes_per_epoch, timesteps=timesteps_per_run )
    mean_ts = np.mean( cartpole_data[ 'timesteps' ] )
    print( f'Average Timesteps: { mean_ts }' )
    
    dq_agent.discount_rate = mean_ts / 25
    
    if dq_agent.discount_rate > 0.8:
        dq_agent.discount_rate = 0.99
    
    if max_time_steps < mean_ts:
        max_time_steps = mean_ts
        torch.save( dq_agent.prediction_model, f'./models/best_model_{mean_ts}-avg.pkl' )
        print( 'Saved new Model' )
    
    X, y = prepare_cartpole_data( cartpole_data )
    
    dataset = CartpoleDataset( X, y )
    data_loader = torch.utils.data.DataLoader( dataset, batch_size=256 )
    for epoch in range( num_epochs ):
        running_loss = 0.0
            
        
        for i, data in enumerate( data_loader ):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.float()
            labels = labels.float()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs =  dq_agent.prediction_model( inputs )
            loss = criterion( outputs, labels )
            running_loss += loss.detach().numpy()
            loss.backward()
            optimizer.step()
        
        print( f'Epoch loss: {running_loss}' )
        
    if (run+1) % epochs_per_update == 0:
        print( 'Updating Target Model' )
        dq_agent.update_target_model()
    


                
                

print('Finished Training')

Average Timesteps: 490.67


NameError: name 'max_time_steps' is not defined

In [ ]:
X.shape

In [51]:
env.close()

In [6]:
dq_agent.prediction_model = torch.load( './models/best_model_483.945-avg.pkl' )
dq_agent.update_target_model()

In [7]:
dq_agent.rand_prob = 0
x = run_episode( env, dq_agent, render=True, timesteps=10000 )
env.close()

In [62]:
test_data[ 0 ]

(array([[-3.17633990e-02,  3.65274766e-02,  3.59507537e-02,
          4.53652801e-02],
        [-3.10328495e-02, -1.59091053e-01,  3.68580593e-02,
          3.49170719e-01],
        [-3.42146705e-02, -3.54717304e-01,  4.38414737e-02,
          6.53244612e-01],
        [-4.13090166e-02, -1.60232387e-01,  5.69063660e-02,
          3.74682817e-01],
        [-4.45136643e-02,  3.40370119e-02,  6.44000223e-02,
          1.00471867e-01],
        [-4.38329241e-02,  2.28179724e-01,  6.64094597e-02,
         -1.71218002e-01],
        [-3.92693296e-02,  4.22291414e-01,  6.29850996e-02,
         -4.42233685e-01],
        [-3.08235013e-02,  2.26337367e-01,  5.41404259e-02,
         -1.30380073e-01],
        [-2.62967540e-02,  3.04833419e-02,  5.15328244e-02,
          1.78880016e-01],
        [-2.56870872e-02,  2.24831422e-01,  5.51104248e-02,
         -9.71111555e-02],
        [-2.11904587e-02,  4.19121981e-01,  5.31682017e-02,
         -3.71910274e-01],
        [-1.28080191e-02,  6.13449872e-01, 